In [391]:
import xgboost as xgb
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [392]:
pd.set_option('display.max_columns',None)
scaler = StandardScaler()

flight_df = pd.read_feather('data/v2_clean_flight')
flight_test_df = pd.read_feather('data/v1_clean_flight_test')
fuel_consumption_df = pd.read_feather('data/v1_clean_fuel_consumption')
passenger_df = pd.read_feather('data/v1_clean_passenger')

In [393]:
def duration_to_rank(x):
    if x == 'short':
        return 0
    elif x =='medium':
        return 1
    else:
        return 2

In [394]:
flight_df['flight_duration'] = flight_df['flight_duration'].apply(lambda x: duration_to_rank(x))

In [395]:
flight_df['day'] = pd.to_datetime(flight_df['fl_date'],yearfirst=True).dt.day
flight_df['month'] = flight_df['month'].astype(int)
flight_df['hour'] = flight_df['hour'].astype(int)

In [396]:
num_cols = ['crs_elapsed_time','distance','month','hour']
cat_cols = ['origin','dest','op_unique_carrier']
y_target = flight_df['arr_delay'].fillna(0)

In [ ]:
flight_model = flight_df[['crs_elapsed_time','distance','month','hour','origin','dest','op_unique_carrier']]
flight_model = pd.get_dummies(flight_model, columns=cat_cols)
flight_model

In [398]:
x_train,x_test,y_train,y_test = train_test_split(flight_model,y_target,test_size=0.3)

In [400]:
#Best parameter is learning_rate 0.1, max_depth 3, n_estimators 50. Took 65minutes to run!, gives r2 of 0.0074

# params = {
#     'max_depth': [3,4,5,6,7],
#     'learning_rate': [0.001,0.01,0.1],
#     'n_estimators': [50,70,100,200]
# }
# xgboost = xgb.XGBRegressor()
# regCV = GridSearchCV(estimator=xgboost,cv=5,param_grid=params)
# regCV.fit(x_train,y_train)
# regCV.best_params_ 

In [401]:
xgboost = xgb.XGBRegressor(objective='reg:squarederror', learning_rate=0.1, max_depth=3, n_estimators=50)
xgboost.fit(x_train,y_train)
y_pred = xgboost.predict(x_test)
r2_score(y_test,y_pred)

0.013376410496370617

In [402]:
#highest is ~0.013 with 'crs_elapsed_time','distance','month','hour','origin','dest','op_unique_carrier'